In [1]:
import warnings
warnings.filterwarnings("ignore")
import os

#To disable all logging output from TensorFlow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import numpy as np
import pandas as pd 
from matplotlib import pyplot as plt
import tensorflow as tf

from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, InputLayer
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder

In [2]:
df=pd.read_csv("transformed_df.csv")

In [3]:
def replace_with_unique(df, column: str):
    uniques = df[column].unique()
    mapping = dict(zip(uniques, range(0, len(uniques))))
    df[column].replace(mapping, inplace=True)
    return df
replace_with_unique(df, "diagnostic")


,age,bleed,grew,itch,elevation,changed,background_mother,background_father,has_sewage_system,region,smoke,gender,pesticide,fitspatrick,hurt,diagnostic,img_id,image_path
0,0.404762,0,0,1,0,0,1,0,0,0,0,1,1,1,0,0,PAT_169_694_411.png,./cancer/all_cancer_images/PAT_169_694_411.png
1,0.511905,0,0,1,0,0,1,1,0,0,0,1,1,1,0,0,PAT_38_1003_68.png,./cancer/all_cancer_images/PAT_38_1003_68.png
2,0.607143,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,PAT_65_101_847.png,./cancer/all_cancer_images/PAT_65_101_847.png
3,0.666667,0,0,1,1,0,1,1,0,0,0,0,0,1,1,0,PAT_69_1053_540.png,./cancer/all_cancer_images/PAT_69_1053_540.png
4,0.809524,1,0,1,1,0,1,1,0,0,0,0,1,1,1,0,PAT_260_400_566.png,./cancer/all_cancer_images/PAT_260_400_566.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,0.726190,0,0,0,0,0,1,1,0,0,1,1,1,0,0,4,PAT_216_331_678.png,./cancer/all_cancer_images/PAT_216_331_678.png
368,0.714286,0,1,1,1,0,1,1,1,0,0,1,0,1,0,4,PAT_365_751_709.png,./cancer/all_cancer_images/PAT_365_751_709.png
369,0.702381,0,1,1,1,1,0,0,1,0,1,1,0,0,0,4,PAT_771_1488_562.png,./cancer/all_cancer_images/PAT_771_1488_562.png
370,0.619048,1,1,1,1,0,1,1,1,0,1,0,0,1,0,4,PAT_319_680_832.png,./cancer/all_cancer_images/PAT_319_680_832.png


In [ ]:
df_naked=df[['region_ABDOMEN', 'region_ARM', 'region_BACK', 'region_CHEST',
       'region_EAR', 'region_FACE', 'region_FOOT', 'region_FOREARM',
       'region_HAND', 'region_LIP', 'region_NECK', 'region_NOSE',
       'region_SCALP', 'region_THIGH', 'age', 'bleed', 'grew', 'itch',
       'elevation', 'changed', 'smoke', 'pesticide', 'hurt', 'diagnostic',]]
df_naked.columns

In [4]:
import numpy as np
X  = np.array(df.drop(df[["diagnostic","img_id", "image_path"]], axis=1))
y = np.array(df["diagnostic"])

In [5]:
X.shape

(372, 15)

In [6]:
y.shape

(372,)

In [7]:
from tensorflow.keras.utils import to_categorical
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y = to_categorical(y)

In [8]:
X[1].shape

(15,)

In [11]:
K.clear_session()
model = Sequential()
model.add(InputLayer(input_shape=X[0].shape))
model.add(Dense(units=15, activation='relu'))
model.add(Dense(units=8, activation='relu'))
model.add(Dense(5,activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 15)                240       
                                                                 
 dense_1 (Dense)             (None, 8)                 128       
                                                                 
 dense_2 (Dense)             (None, 5)                 45        
                                                                 
Total params: 413
Trainable params: 413
Non-trainable params: 0
_________________________________________________________________


In [12]:
from tensorflow import keras
#model.compile(optimizer=Adam(learning_rate=0.01), loss=keras.losses.categorical_crossentropy, metrics=[keras.metrics.categorical_accuracy])
opt = tf.keras.optimizers.RMSprop()
model.compile(loss='categorical_crossentropy',optimizer= opt, metrics=['accuracy'])
history = model.fit(X, y, batch_size=32, epochs=20_000, validation_split=0.2)

Epoch 1/20000
10/10 [==============================] - 0s 15ms/step - loss: 1.5999 - accuracy: 0.2020 - val_loss: 1.8925 - val_accuracy: 0.0000e+00
Epoch 2/20000
10/10 [==============================] - 0s 3ms/step - loss: 1.5735 - accuracy: 0.2155 - val_loss: 1.9384 - val_accuracy: 0.0000e+00
Epoch 3/20000
10/10 [==============================] - 0s 3ms/step - loss: 1.5538 - accuracy: 0.2593 - val_loss: 1.9949 - val_accuracy: 0.0000e+00
Epoch 4/20000
10/10 [==============================] - 0s 3ms/step - loss: 1.5350 - accuracy: 0.2660 - val_loss: 2.0395 - val_accuracy: 0.0000e+00
Epoch 5/20000
10/10 [==============================] - 0s 3ms/step - loss: 1.5167 - accuracy: 0.2795 - val_loss: 2.0944 - val_accuracy: 0.0000e+00
Epoch 6/20000
10/10 [==============================] - 0s 3ms/step - loss: 1.4992 - accuracy: 0.3064 - val_loss: 2.1501 - val_accuracy: 0.0000e+00
Epoch 7/20000
10/10 [==============================] - 0s 3ms/step - loss: 1.4800 - accuracy: 0.3098 - val_loss: 2.21

KeyboardInterrupt: 

In [ ]:
history.history.keys()

In [ ]:
pd.DataFrame(history.history["val_categorical_accuracy"]).plot()

In [ ]:
model.save("model_moons.h5")

In [ ]:
ypred = model.predict(X)
plt.scatter(X[:, 0], X[:, 1], c=ypred[:, 0])